In [6]:
!pip install tqdm
!pip install torch
!pip install torchvision torchaudio
!pip install fastai
!pip install tensorboardX
!pip install scikit-learn
!pip install "git+https://github.com/ncullen93/torchsample.git#egg=torchsample"
!pip install nibabel
!pip install nbdev
!pip install torch==1.6.0 torchvision==0.7.0

     |████████████████████████████████| 1.5 MB 4.8 MB/s eta 0:00:01


  Cloning https://github.com/ncullen93/torchsample.git to /private/var/folders/tt/s_pztz81715_h0sd9m9nsllw0000gn/T/pip-install-r7hrl2o2/torchsample_73f2f124c4d04b13b2ea46e2db961757
  Running command git clone -q https://github.com/ncullen93/torchsample.git /private/var/folders/tt/s_pztz81715_h0sd9m9nsllw0000gn/T/pip-install-r7hrl2o2/torchsample_73f2f124c4d04b13b2ea46e2db961757
  Created wheel for torchsample: filename=torchsample-0.1.3-py3-none-any.whl size=43415 sha256=5b298480751f126e21cdf3866fc2464b52d4c04b9b57a6b474ca608aed272130
  Stored in directory: /private/var/folders/tt/s_pztz81715_h0sd9m9nsllw0000gn/T/pip-ephem-wheel-cache-lrkx_l_r/wheels/54/14/cd/5612ec3bc0ce8e5b68b6a273873d18a3a5c817f35db70579a1
Successfully built torchsample
     |████████████████████████████████| 3.3 MB 6.1 MB/s eta 0:00:01


     |████████████████████████████████| 97.5 MB 15.6 MB/s eta 0:00:01
     |████████████████████████████████| 387 kB 2.1 MB/s eta 0:00:01
  Using cached future-0.18.2-py3-none-any.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.8.1
    Uninstalling torch-1.8.1:
      Successfully uninstalled torch-1.8.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.9.1
    Uninstalling torchvision-0.9.1:
      Successfully uninstalled torchvision-0.9.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.8.1 requires torch==1.8.1, but you have torch 1.6.0 which is incompatible.
fastai 2.3.1 requires torch<1.9,>=1.7.0, but you have torch 1.6.0 which is incompatible.
fastai 2.3.1 requires torchvision>=0.8.2, but you have torchvision 0.7.0 which is incompatible.


In [7]:
import shutil
import os
import time
from datetime import datetime
import argparse
import numpy as np
from tqdm import tqdm
from nbdev import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from dataloader import MRDataset
import model

from sklearn import metrics
from fastai.vision.all import *

ModuleNotFoundError: No module named 'nbdev'

In [28]:
import multiprocessing

num_workers = multiprocessing.cpu_count() - 1
num_workers

23

In [29]:
class Args:
    def __init__(self):
        self.task = "abnormal" #['abnormal', 'acl', 'meniscus']
        self.plane = "sagittal" #['sagittal', 'coronal', 'axial']
        self.prefix_name = "Test"
        self.augment = 1 #[0, 1]
        self.lr_scheduler = "plateau" #['plateau', 'step']
        self.gamma = 0.5
        self.epochs = 1
        self.lr = 1e-5
        self.flush_history = 0 #[0, 1]
        self.save_model = 1 #[0, 1]
        self.patience = 5
        self.log_every = 100
        
args = Args()

In [30]:
log_root_folder = "./logs/{0}/{1}/".format(args.task, args.plane)
if args.flush_history == 1:
    objects = os.listdir(log_root_folder)
    for f in objects:
        if os.path.isdir(log_root_folder + f):
            shutil.rmtree(log_root_folder + f)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)

augmentor = Compose([
    transforms.Lambda(lambda x: torch.Tensor(x)),
    RandomRotate(25),
    RandomTranslate([0.11, 0.11]),
    RandomFlip(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1, 1).permute(1, 0, 2, 3)),
])

train_dataset = MRDataset('./data/', args.task, args.plane, transform=augmentor, train=True)
validation_dataset = MRDataset('./data/', args.task, args.plane, train=False)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=num_workers, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False, num_workers=num_workers, drop_last=False)


mrnet = model.MRNet()
bceloss = nn.BCEWithLogitsLoss()

./data/train-abnormal.csv


In [31]:
dls = DataLoaders(train_loader, validation_loader)

In [32]:
metrics = [RocAuc()]

learn = Learner(dls, mrnet, loss_func=bceloss, metrics=metrics)

In [33]:
learn.fine_tune(2)

In [34]:
learn.save(args.task[:3] + '_' + args.plane[:3] + '_v1')

Path('models/abn_sag_v1.pth')